# N-Gram Language Models #

Language models are one of the most important concepts in NLP. They help us assign probabilities to sequence of words, such as sentences or phrases. As we saw in the previous lecture their application spans beyond NLP. For example, language models help us obtain better speech recognition, optical character recognition (OCR) and information retrieval results, to name a few. Given a sentence $S$ with a set of $w_i$ words, $i=1,2,...,n$, language models formally define the probability of the sentence as the probability of having the particular sequence of words:  
$$ \Large p(S)=p(w_1, w_2, w_3, ... , w_n) $$  

They also help us predict the probability of a specific word given the previous words in the sentence:  

$$ \Large p(w | w_{-1}, w_{-2}..,w_{-k}) $$  

The probability of a sentence is computed using the chain rule:  

$$ \Large p(S)=p(w_1, w_2, w_3, ... , w_n) = \prod_i^n {p(w_i|w_1,w_2,...,w_{i-1})} $$  

When computing this probability we use the Markov assumption which simplifies the computation of the above probability:    

$$ \Large p(S)=p(w_1, w_2, w_3, ... , w_n) = \prod_i^n {p(w_i|w_{i-k},w_{i-(k-1)},...,w_{i-1})} $$  

The Markov assumption states that the conditional probability of a word $w_i$, given all of the previous words in the sentence, could be approximated by considering only the $k$ previous words.


### N-Gram Models ###

#### Unigram Model ####
The unigram language model is the simplest of the n-gram models. Under this model the probability of the sentence $S$ is simply a product of the probabilities of each individual word in the sentence:  

$$ \Large p(S) = \prod_i^n {p(w_i)} $$  

N-gram probabilities are computed using Maximum Likelihood Estimates (MLE). For the unigram language model we compute MLE by counting the number of times word $w_i$ occurred in the collection and we divide that number by the total number of words $v$ in the collection:
$$ \Large p(w_i) = \frac{count(w_i)}{\sum_{i=1}^{v}{count(w_i)}} $$  

#### Bigram Model ####
In the bigram model the probability of a word is conditioned only on the previous word:  
$$ \Large p(S) = \prod_i^n {p(w_i|w_{i-1})} $$  

The MLE for the bigram LM is computed by dividing the number of times words $w_i$ and $w_{i-1}$ occurred together in the collection with the number of occurrences of word $w_i$, $count(w_{i-1})$ :

$$ \Large p(w_i|w_{i-1}) = \frac{count(w_{i-1},w_i)}{count(w_{i-1})} $$  

#### Trigram Model ####
In the trigram model the probability of a word is conditioned on the previous two words:  
$$ \Large p(S) = \prod_i^n {p(w_i|w_{i-2},w_{i-1})} $$  

The MLE for the trigram LM is computed by dividing the number of times words $w_i$, $w_{i-1}$, $w_{i-2}$ occurred together in the collection with the count for $w_{i-1}$ and $w_{i-2}$ occurring together, $count(w_{i-1},w_{i-2})$:

$$ \Large p(w_i|w_{i-1},w_{i-2}) = \frac{count(w_{i-2},w_{i-1},w_i)}{count(w_{i-2},w_{i-1})} $$  

## Computing Bigrams ##
In this task we are going to compute unigram and bigram language models over a collection of news stories from various news agencies that were published by the Associated Press Worldstream English Service on September 30th 2010. Rather than implementing language models on our own we are going to use the nltk package. Together let's go over the code step by step.

We'll first load the NLTK stopwords list and also define an array of punctuation marks:

In [ ]:
import nltk
import string
import itertools
#load the NLTK stopwords list:
stopwords_list = nltk.corpus.stopwords.words('english')

Next we are going to read the collection of news stories. This collection is stored in a tsv file where each news story is stored into a single line. In each line the number of tsv fields varies but the first three fields always contain the same type of an information. These fields are:  
__article_id__, __location with date and time__ and the __news story title__.  
Here is an example:  
APW_ENG_20100930.0015__\t__TOKYO 2010-09-30 00:10:50 UTC__\t__Japan factory output down for third straight month  
The remaining tab separated fields within the line contain the news story sentences.  
One way to load this data would be through pandas but due to the variable number of tsv fields across lines we wouldn't be able to do that. Therefore we'll use the traditional approach of reading a file line by line. 

In [ ]:
newsstories = "../../../data/gigaword_20b_1k"

articles = open(newsstories,'r')
all_words = list()
a_count=0
#Create a translation table:
translator=str.maketrans('','',string.punctuation)

tokenized_sentences = list()
for article in articles.readlines():
    article = article.strip()
    a_count += 1
    if (a_count % 100 == 0):
        print(a_count)
    fields = article.split("\t")
    article_id = fields[0]
    place = fields[1].split(" ")[0]
    date = fields[1].split(" ")[1]
    time = fields[1].split(" ")[2]
    title = fields[2]
    
    article_content = fields[2::]
    for sent in article_content:
        tokenized_sent = list()
        sent_words = nltk.word_tokenize(sent)
        sent_words = [word for word in sent_words if ((len(word) > 1) and (len(word) < 20))]
        sent_words = [word for word in sent_words if word not in stopwords_list]
        for word in sent_words:
            word = word.lower()
            word=word.translate(translator)
            all_words.append(word)
            tokenized_sent.append(word)
        tokenized_sentences.append(tokenized_sent)
        print(tokenized_sent)    

Next we will compute the bigram probabilities:

In [ ]:
#First compute the bigrams (i.e. the tuple of words that occur together)
bigram_model  = nltk.bigrams(all_words)

#Then compute the frequency for each bigram
bigram_frequency_word = nltk.ConditionalFreqDist(bigram_model)
#To get the bigram probabilities we would need to normalize the bigram frequencies:
bigram_probability_word = nltk.ConditionalProbDist(bigram_frequency_word, nltk.MLEProbDist)

#Let's compute the bigram probabilities constraining on sentences:
bigram_frequency_sent = nltk.ConditionalFreqDist((word[0],word[1]) for word in list( itertools.chain (*[nltk.bigrams(sent) for sent in tokenized_sentences])))
bigram_probability_sent = nltk.ConditionalProbDist(bigram_frequency_sent, nltk.MLEProbDist)

#Let's print the bigram probabilities computed over sentences:
bigram_probability = bigram_probability_sent
bigram_frequency = bigram_frequency_sent

all_bigrams = dict()
for source_word in bigram_probability:
	prob_words = bigram_probability[source_word].samples()
	denom = len(prob_words)
	all_bigrams[source_word]=dict()
	for target_word in prob_words:
		prob = bigram_probability[source_word].prob(target_word)
		all_bigrams[source_word][target_word] = prob
		print ("p("+target_word+"|"+source_word+")={0:.4f}".format(prob))

Now that we've generated the bigram language model we could make query about any bigram and obtain its probability using the following code:  
bigram_probability[$w_{i-1}$].prob($w_{i}$)  
Where $w_{i-1}$ is the prior word and $w_{i}$ is the current word. For example the code below would give us the probability of the bigram "fiat ceo"

In [ ]:
print (bigram_probability["fiat"].prob("ceo"))

For a given word in our collection we could also obtain the probability of all of its bigrams. For example the code below gives us all the possible bigrams for the word "september"

In [ ]:
all_bigrams["september"]

**[Assignment 1]**  
Use the above code to explore the bigram model that we generated.

## Computing Unigrams##

In our lab session on Tuesday we used the __nltk.FreqDist__ method to obtain the frequency count over all the words in our collection. The input argument to this method is the list of words in the collection. This method returns a dictionary where the keys are the words in the collection and the values are their frequency counts. For example if the word __car__ occurred 10 times in our collection the dictionary value for this word would be 10: __dict["car"]=10__.

**[Assignment 2]**  
In this part of the lab session you are asked to compute the unigram probabilities using the __nltk.FreqDist__ method. At the beginning of this lab session we stored all words in the collection in the __all_words__ list. To compute the unigram probability for each word in the collection you would need to divide the frequency count of that word with the total number of word instances in the collection which is equal to the sum of all frequency counts. 
Provided below is the code framework that you could use for this task. In order to compute the unigram probabilities you would need to implement the missing code.

In [ ]:
unigram_frequency = nltk.FreqDist(all_words)
#2a: Compute the total number of word instances in the collection. This is the sum of all word frequencies. 
sum_freq = ?
#Use the following dictionary to store the unigram probabilities for each word:
all_unigrams = dict()
#In order to compute the unigram probability for each word you would need to 
#iterate over all the words in the collection:
for word in unigram_frequency:
    #2b: Obtain the word frequency count:
    freq = ?
    #2c: Compute the probability:
    prob = (1.0)*freq/?
    #Store the probability value into our dictionary. 
    all_unigrams[word]= prob
    print ("p("+word+")={0:.4f}".format(prob))

**[Assignment 3]**  
Now that we've computed the unigram probabilities you could obtain the probability for each word in the collection by querying the __all_unigrams__ dictionary.